In [10]:
import pandas as pd
import numpy as np

In [11]:
filepath = './HeroesOfPymoli_Resources_purchase_data.csv'

In [12]:
df = pd.read_csv(filepath,encoding="ISO-8859-1")

In [13]:
#preview of DataFrame
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [14]:
#identify complete rows
df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [16]:
#review the column types
df.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [27]:
#To confirm a unique Item ID per Item Name
df.groupby('Item Name')['Item ID'].nunique()

Item Name
Abyssal Shard                               1
Aetherius, Boon of the Blessed              1
Agatha                                      1
Alpha                                       1
Alpha, Oath of Zeal                         1
Alpha, Reach of Ending Hope                 1
Amnesia                                     1
Apocalyptic Battlescythe                    1
Arcane Gem                                  1
Avenger                                     1
Azurewrath                                  1
Betrayal, Whisper of Grieving Widows        1
Betrayer                                    1
Blade of the Grave                          1
Blazefury, Protector of Delusions           1
Blazeguard, Reach of Eternity               1
Blindscythe                                 1
Blood Infused Guardian                      1
Blood-Forged Skeletal Spine                 1
Bloodlord's Fetish                          1
Bone Crushing Silver Skewer                 1
Bonecarvin Battle Axe   

In [ ]:
#source[unqiue values per group]:https://stackoverflow.com/questions/41415017/count-unique-values-using-pandas-groupby

In [60]:
#to count the diversity of ages per gender 
df_demographicdiversity= df.groupby('Gender')['Age'].nunique()
df_demographicdiversity

Gender
Female                   22
Male                     39
Other / Non-Disclosed     8
Name: Age, dtype: int64

In [50]:
#to count the number of Items ID purchased per SN
df_numofitemspur= df.groupby('SN')['Item ID'].nunique().sort_values(ascending = False).reset_index(name= "Total Purchased Products")
df_numofitemspur

,SN,Total Purchased Products
0,Lisosia93,5
1,Iral74,4
2,Idastidru52,4
3,Aina42,3
4,Iri67,3
5,Chamjask73,3
6,Iskadarya95,3
7,Phyali88,3
8,Chanastnya43,3
9,Silaera56,3


In [ ]:
#source[Name a df column]: https://stackoverflow.com/questions/29836477/pandas-create-new-column-with-count-from-groupby

In [59]:
df_numofitemsbySN= df_numofitemspur.groupby('Total Purchased Products').count().sort_values(by="SN", ascending = False)
df_numofitemsbySN

,SN
Total Purchased Products,
1,414
2,124
3,35
4,2
5,1


In [61]:
#statistic summary on Gender and Age
df_statsbyGenderAge= round(df.groupby('Gender')['Age'].describe().reset_index(),0)
df_statsbyGenderAge

,Gender,count,mean,std,min,25%,50%,75%,max
0,Female,113.0,21.0,6.0,7.0,18.0,21.0,24.0,40.0
1,Male,652.0,23.0,7.0,7.0,20.0,22.0,25.0,45.0
2,Other / Non-Disclosed,15.0,24.0,8.0,15.0,18.0,21.0,30.0,38.0


In [34]:
#source[summarystatisticsbygroup]:https://datascience.stackexchange.com/questions/22266/summary-statistics-by-category-using-python

In [62]:
#statistic summary on item name and prices
df_statsbyItem =round(df.groupby('Item Name')['Price'].describe().reset_index(),2)
df_statsbyItem 

,Item Name,count,mean,std,min,25%,50%,75%,max
0,Abyssal Shard,5.0,2.67,0.00,2.67,2.67,2.67,2.67,2.67
1,"Aetherius, Boon of the Blessed",5.0,3.39,0.00,3.39,3.39,3.39,3.39,3.39
2,Agatha,6.0,3.08,0.00,3.08,3.08,3.08,3.08,3.08
3,Alpha,3.0,2.07,0.00,2.07,2.07,2.07,2.07,2.07
4,"Alpha, Oath of Zeal",3.0,4.05,0.00,4.05,4.05,4.05,4.05,4.05
5,"Alpha, Reach of Ending Hope",1.0,3.58,NaN,3.58,3.58,3.58,3.58,3.58
6,Amnesia,6.0,2.18,0.00,2.18,2.18,2.18,2.18,2.18
7,Apocalyptic Battlescythe,6.0,1.97,0.00,1.97,1.97,1.97,1.97,1.97
8,Arcane Gem,3.0,3.79,0.00,3.79,3.79,3.79,3.79,3.79
9,Avenger,6.0,3.44,0.00,3.44,3.44,3.44,3.44,3.44


In [ ]:
#statistic summary on Gender and Age